In [151]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [152]:
POSSIBLE_VALUES = [2,3,4,5,6,7,8,9,10,11]

DECK_COLUMNS = [str(i) for i in POSSIBLE_VALUES]
HAND_COLUMNS = ["chance", "score", "cards", "aces"] + DECK_COLUMNS

In [153]:
def get_deck(number_decks=6):
    deck = pd.Series([number_decks*4]*len(DECK_COLUMNS) ,index=DECK_COLUMNS)
    deck.loc["10"] *= 4
    return deck

In [154]:
def update_hand_with_card(hand, value, probabilistic=True, update_deck=True):
    new_possibles = hand.copy()

    assert all(hand["score"] < 22) and all(hand["score"] != -1), "there are finished hands"

    if probabilistic:
        # updating chance: multiplying by the probability of drawing that value
        new_possibles["chance"] *= (new_possibles[f"{value}"]/new_possibles[DECK_COLUMNS].sum(axis=1))
    if update_deck:
        # update deck info
        new_possibles[f"{value}"] -= 1

    # updating score
    new_possibles["score"] += value
    # updating cards
    new_possibles["cards"] += 1
    # updating aces
    new_possibles["aces"] += (value == 11)

    # if necessary, using ace as 1 instead of 11
    acenecessary = (new_possibles["score"] > 21) & (new_possibles["aces"] > 0)
    new_possibles.loc[acenecessary,"score"] -= 10
    new_possibles.loc[acenecessary,"aces"] -= 1

    # set bust to -1
    new_possibles.loc[new_possibles["score"] > 21, "score"] = -1 

    # set blackjack to 22
    new_possibles.loc[(new_possibles["score"] == 21) & (new_possibles["cards"] == 2), "score"] = 22


    return new_possibles

In [155]:
def get_hand(deck=None, card_values=[], update_deck=True):
    if deck is None:
        deck = get_deck()
    hand = pd.DataFrame([1.0, 0, 0, 0] + deck.values.flatten().tolist(), index=HAND_COLUMNS).T.astype(int).astype({"chance":float})

    for card_value in card_values:
        assert np.all(hand["score"] != -1), f"hand is busted, but tried to add card {card_value}"

        hand = update_hand_with_card(hand, card_value, probabilistic=False, update_deck=update_deck)
    
    return hand

In [176]:
def get_hand_hitting_once(hand):
    """get updated hand when hitting once"""

    assert type(hand) == pd.DataFrame and hand.columns.tolist() == HAND_COLUMNS, f"hand is not valid, got {hand}"

    finished = (hand["score"] == -1) | (hand["score"] == 22)
    if finished.sum() == len(hand):
        print("All possible hands are finished")
        return hand
    elif finished.sum() > 0:
        print(f"{finished.sum()} possible hands are finished already")
    
    outcome_hand = pd.DataFrame(hand[finished], columns=HAND_COLUMNS)
    hand = hand[~finished]

    for possible_value in POSSIBLE_VALUES:
        new_possibles = update_hand_with_card(hand, possible_value)
        
        # adding to all possible outcomes
        outcome_hand = pd.concat((outcome_hand, new_possibles), axis=0, ignore_index=True)
    
    return outcome_hand.groupby(["score","cards","aces"] + DECK_COLUMNS).agg({'chance': 'sum'}).reset_index()[HAND_COLUMNS].sort_values("score")

In [ ]:
def get_dealer_score_probabilities(dealer_hand):
    """get probabilities of each possible score given the dealers current hand"""

    assert type(dealer_hand) == pd.DataFrame and dealer_hand.columns.tolist() == HAND_COLUMNS, f"hand is not valid, got {dealer_hand}"
    
    is_playable = (dealer_hand["score"] < 17) & (dealer_hand["score"] != -1)
    final = dealer_hand[~is_playable]
    playable = dealer_hand[is_playable]

    while len(playable) > 0:
        new_dealer_hand = get_hand_hitting_once(playable)
        final = pd.concat((final, new_dealer_hand[~(new_dealer_hand["score"] < 17) & (new_dealer_hand["score"] != -1)]), axis=0, ignore_index=True)
        playable = new_dealer_hand[(new_dealer_hand["score"] < 17) & (new_dealer_hand["score"] != -1)]
    
    scores = get_score_probabilities(final)
    
    return scores

In [177]:
def get_score_probabilities(hand):
    chances = hand.groupby(["score"]).agg({'chance': 'sum'}).reset_index().sort_values("score")
    return pd.Series(index=chances["score"].values, data=chances["chance"].values)

In [228]:
deck = get_deck()
hand = get_hand(deck, [])

In [233]:
hand = get_hand_hitting_once(hand)
hand

42 possible hands are finished already


,chance,score,cards,aces,2,3,4,5,6,7,8,9,10,11
0,0.021830,-1,3,0,23,24,24,24,24,24,24,24,94,24
247,0.000125,-1,4,0,24,24,24,21,24,23,24,24,96,24
246,0.002826,-1,4,0,24,24,23,24,24,24,24,24,94,23
245,0.001999,-1,4,0,24,24,23,24,24,24,24,23,95,23
244,0.000308,-1,4,0,24,24,23,24,24,24,24,22,96,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,0.000857,21,4,0,24,23,23,24,23,24,23,24,96,24
671,0.000410,21,4,0,24,23,23,24,24,22,24,24,96,24
672,0.000410,21,4,0,24,23,24,22,24,24,23,24,96,24
661,0.000410,21,4,0,23,24,24,23,24,22,24,24,96,24


In [234]:
scores = get_score_probabilities(hand)
scores

-1     0.734243
 8     0.000027
 9     0.000125
 10    0.000322
 11    0.000661
 12    0.004861
 13    0.007713
 14    0.010477
 15    0.013852
 16    0.017832
 17    0.022276
 18    0.027156
 19    0.032290
 20    0.037642
 21    0.043033
 22    0.047489
dtype: float64

In [236]:
deck = get_deck()
hand = get_hand(deck, [])

In [237]:
dealer_score = get_dealer_score_probabilities(hand)
dealer_score

17    0.145245
18    0.139258
19    0.133685
20    0.179527
21    0.072874
22    0.047489
dtype: float64

In [163]:
def get_expected_return_standing(deck, player_hand, dealer_hand):
    """Returns the expected return of standing with the given cards."""

    assert type(player_hand) == pd.DataFrame and player_hand.columns.tolist() == HAND_COLUMNS, f"player_hand is not valid, got {player_hand}"
    assert type(dealer_hand) == pd.DataFrame and dealer_hand.columns.tolist() == HAND_COLUMNS, f"dealer_hand is not valid, got {dealer_hand}"
    assert all(dealer_hand["cards"] == 1), "Dealer must have exactly one card showing."

    print("Calculating expected return of standing with the following hand:")
    print("player", player_hand)
    print("dealer", dealer_hand)


    dealer_score_probabilities = get_dealer_score_probabilities(deck, dealer_hand)
    # TODO: this is actually incorrect, as it does not take into account the card just drawn by the player
    
    player_score_probabilities = get_score_probabilities(player_hand)

    print("Dealer score probabilities:")
    print(dealer_score_probabilities)

    win, push, lose = 0.0, 0.0, 0.0

    for score, chance in player_score_probabilities.items():
        win += chance * dealer_score_probabilities[dealer_score_probabilities.index < score].sum()
        push += chance * dealer_score_probabilities[dealer_score_probabilities.index == score].sum()
        lose += chance * dealer_score_probabilities[dealer_score_probabilities.index > score].sum()

    print(pd.DataFrame([win,push,lose], index=["win","push","lose"], columns=["probability"]).T)
    print("Expected return of standing is", win - lose)

    return win - lose

In [164]:
def get_expected_return_hitting_once(deck, player_hand, dealer_hand):
    """Returns the expected return of hitting exactly once with the given cards."""
    
    assert type(player_hand) == pd.DataFrame and player_hand.columns.tolist() == HAND_COLUMNS, f"player_hand is not valid, got {player_hand}"
    assert type(dealer_hand) == pd.DataFrame and dealer_hand.columns.tolist() == HAND_COLUMNS, f"dealer_hand is not valid, got {dealer_hand}"
    assert all(dealer_hand["cards"] == 1), "Dealer must have exactly one card showing."

    print("Calculating expected return of standing with the following hand:")
    print("player", player_hand)
    print("dealer", dealer_hand)


    dealer_score_probabilities = get_dealer_score_probabilities(deck, dealer_hand)
    # TODO: this is actually incorrect, as it does not take into account the card just drawn by the player

    player_hand = get_hand_hitting_once(player_hand)
    player_score_probabilities = get_score_probabilities(player_hand)

    print("Dealer score probabilities:")
    print(dealer_score_probabilities)

    win, push, lose = 0.0, 0.0, 0.0

    for score, chance in player_score_probabilities.items():
        win += chance * dealer_score_probabilities[dealer_score_probabilities.index < score].sum()
        push += chance * dealer_score_probabilities[dealer_score_probabilities.index == score].sum()
        lose += chance * dealer_score_probabilities[dealer_score_probabilities.index > score].sum()

    print(pd.DataFrame([win,push,lose], index=["win","push","lose"], columns=["probability"]).T)
    print("Expected return of hitting once is", win - lose)

    return win - lose

In [238]:
deck = get_deck()
player = get_hand(deck, [10,6])
dealer = get_hand(deck, [7])

r_stand = get_expected_return_standing(deck, player, dealer)
r_hit = get_expected_return_hitting_once(deck, player, dealer)

print("")
print("Expected return of standing is", r_stand)
print("Expected return of hitting once is", r_hit)

Calculating expected return of standing with the following hand:
player    chance  score  cards  aces   2   3   4   5   6   7   8   9  10  11
0     1.0     16      2     0  24  24  24  24  23  24  24  24  95  24
dealer    chance  score  cards  aces   2   3   4   5   6   7   8   9  10  11
0     1.0      7      1     0  24  24  24  24  24  23  24  24  96  24


TypeError: get_dealer_score_probabilities() takes 1 positional argument but 2 were given

In [ ]:
#TODO: implement chained card probabilities (player draws before dealer, so dealer's probabilities should be conditional on player's card)
#TODO: implement splitting
#TODO: implement doubling down